In [1]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 63.9 MB/s 
     |████████████████████████████████| 120 kB 57.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.0 MB/s 
     |████████████████████████████████| 115 kB 60.8 MB/s 
     |████████████████████████████████| 212 kB 49.3 MB/s 
     |████████████████████████████████| 127 kB 63.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 8.9 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.utils import shuffle
import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [5]:
pd.set_option('display.max_rows', None)

In [6]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('ERAI_Dataset_pairwise_test_with_translation.json')
df.reset_index(drop = True,inplace = True)
df = df.dropna(axis = 0)

df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))
df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))


In [7]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"


tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [8]:



def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.float),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.float),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [9]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes_cls,n_classes_reg):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("bert-base-chinese")
    self.drop = nn.Dropout(p=0.1)
    self.out_cls = nn.Linear(self.bert.config.hidden_size, n_classes_cls)

    self.out_reg = nn.Linear(self.bert.config.hidden_size, n_classes_reg)
    self.sigmoid = nn.Sigmoid()

  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    cls_output = []
    sent1_output = []
    sent2_output = []
    
    output = self.drop(output[0])

    for last_output,len1,len2 in zip(output,length1,length2):
      cls_output.append(last_output[0])
      
      sent1_output.append(torch.mean(last_output[1:(len1+1)],dim=0))
      sent2_output.append(torch.mean(last_output[len1+2:(len1+2+len2)],dim=0))

    cls_output = torch.stack(cls_output)
    sent1_output = torch.stack(sent1_output)
    
    sent2_output = torch.stack(sent2_output)
    
    cls_output = self.out_cls(cls_output)
    
    
    
    return self.sigmoid(cls_output),self.out_reg(sent1_output),self.out_reg(sent2_output)

In [22]:
a = torch.randint(0,10,(8,))
b = torch.randint(0,10,(8,))
print(a)
print(b)

tensor([3, 3, 2, 8, 1, 4, 5, 4])
tensor([9, 9, 0, 1, 6, 0, 0, 1])


In [24]:
mpp_ml_label = torch.stack((a,b),1)
mpp_ml_label

tensor([[3, 9],
        [3, 9],
        [2, 0],
        [8, 1],
        [1, 6],
        [4, 0],
        [5, 0],
        [4, 1]])

In [10]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [11]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [25]:
EPOCHS = 50

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_reg = nn.MSELoss().to(device)
loss_cls = nn.BCELoss().to(device)



In [26]:

def train_and_eval(model, 
  train_loader, 
  validation_loader, 
  optimizer,  
  device,
  epoch,
  # label_num 
  ):
  model.train()
  max_len = 256
  current_accuracy_state = 0
  for epo in range(epoch):
    
    train_loss = []
    
    global train_mpp_predictions 
    train_mpp_predictions = []
    global train_ml_predictions
    train_ml_predictions = []
    

    validation_loss = []
    
    validation_mpp_predictions = []
    validation_ml_predictions = []


    global real_train_mpp 
    real_train_mpp = []
    global real_train_ml
    real_train_ml = []
    real_validation_mpp = []
    real_validation_ml = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      output_cls,output_sen1,output_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      
      mpp_ml_label = torch.stack((mpp_targets,ml_targets),1)
     
      loss1 = loss_cls(output_cls.view(-1),mpp_ml_label.view(-1)).to(device)
      
    
      
      loss = loss1
 

      train_loss.append(loss.item())


      for i in output_cls:
        if i[0]>0.5:
          train_mpp_predictions.append(1)
        else:
          train_mpp_predictions.append(0)
        if i[1]>0.5:
          train_ml_predictions.append(1)
        else:
          train_ml_predictions.append(0)

      for i,j in zip(mpp_targets,ml_targets):
        real_train_mpp.append(i.cpu().item())
        real_train_ml.append(j.cpu().item())

      
    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
        
   
        )
        mpp_ml_label = torch.stack((mpp_targets,ml_targets),1)
     
        loss1 = loss_cls(output_cls.view(-1),mpp_ml_label.view(-1)).to(device)
        
     
        loss = loss1
        

        validation_loss.append(loss.item())
      

        for i in output_cls:
          if i[0]>0.5:
            validation_mpp_predictions.append(1)
          else:
            validation_mpp_predictions.append(0)
          if i[1]>0.5:
            validation_ml_predictions.append(1)
          else:
            validation_ml_predictions.append(0)
   

        for i,j in zip(mpp_targets,ml_targets):
          real_validation_mpp.append(i.cpu().item())
          real_validation_ml.append(j.cpu().item())
    
    train_mpp_accuracy = accuracy_score(real_train_mpp, train_mpp_predictions)
    train_ml_accuracy = accuracy_score(real_train_ml, train_ml_predictions)
    validation_mpp_accuracy = accuracy_score(real_validation_mpp, validation_mpp_predictions)
    validation_ml_accuracy = accuracy_score(real_validation_ml, validation_ml_predictions)
    
    final_train_loss = np.mean(train_loss)
 
    final_validation_loss = np.mean(validation_loss)

    average_accuracy = (validation_mpp_accuracy+validation_ml_accuracy)/2

    if average_accuracy > current_accuracy_state:
      current_accuracy_state =  average_accuracy
      torch.save(model.state_dict(), 'double_value_cls.bin')



    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train MPP Accuracy: {:.4f}, Train ML Accuracy: {:.4f}, Valid Total Loss: {:.4f}, Valid MPP Accuracy: {:.4f}, Valid ML Accuracy: {:.4f}'
    .format(epo+1,final_train_loss, train_mpp_accuracy, train_ml_accuracy,final_validation_loss,validation_mpp_accuracy,validation_ml_accuracy))

In [27]:
random_seed = [171,354,550,667,985]
for i in random_seed:
  
  df = shuffle(df,random_state=i)

  
  train = df[0:156]

  test = df[156:200]

  train_post1 = train['post1']
  train_post2 = train['post2']
  train_mpp_label = train['MPP_label']
  train_ml_label = train['ML_label']
  train_mpp_value1 = train['MPP1'] 
  train_mpp_value2 = train['MPP2']
  train_ml_value1 = train['ML1']
  train_ml_value2 = train['ML2']

  test_post1 = test['post1']
  test_post2 = test['post2']
  test_mpp_label = test['MPP_label']
  test_ml_label = test['ML_label']
  test_mpp_value1 = test['MPP1'] 
  test_mpp_value2 = test['MPP2']
  test_ml_value1 = test['ML1']
  test_ml_value2 = test['ML2']

  max_len = 256
  batch_size = 8
  train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
  test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)

  label_num_cls = 2
  label_num_reg = 2
  model= BertNerClassifier(label_num_cls,label_num_reg).to(device)
  optimizer = AdamW(model.parameters(), lr=2e-4, correct_bias=False)
  
  
  train_and_eval(model, 
    train_dataloader, 
    test_dataloader, 
    optimizer,  
    device,
    EPOCHS,
    )




Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.9373, Train MPP Accuracy: 0.4872, Train ML Accuracy: 0.4744, Valid Total Loss: 0.7907, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.6000
Epoch [2] Train Totol Loss: 1.0864, Train MPP Accuracy: 0.4744, Train ML Accuracy: 0.4679, Valid Total Loss: 0.7046, Valid MPP Accuracy: 0.6250, Valid ML Accuracy: 0.5500
Epoch [3] Train Totol Loss: 0.7269, Train MPP Accuracy: 0.5128, Train ML Accuracy: 0.5192, Valid Total Loss: 0.6921, Valid MPP Accuracy: 0.6250, Valid ML Accuracy: 0.4500
Epoch [4] Train Totol Loss: 0.7182, Train MPP Accuracy: 0.4744, Train ML Accuracy: 0.5064, Valid Total Loss: 0.6810, Valid MPP Accuracy: 0.6250, Valid ML Accuracy: 0.4500
Epoch [5] Train Totol Loss: 0.7059, Train MPP Accuracy: 0.4744, Train ML Accuracy: 0.5064, Valid Total Loss: 0.6881, Valid MPP Accuracy: 0.6250, Valid ML Accuracy: 0.4500
Epoch [6] Train Totol Loss: 0.7028, Train MPP Accuracy: 0.4744, Train ML Accuracy: 0.4808, Valid Total Loss: 0.6912, Valid MPP Accuracy: 0.6250, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.9639, Train MPP Accuracy: 0.5321, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6890, Valid MPP Accuracy: 0.5750, Valid ML Accuracy: 0.5750
Epoch [2] Train Totol Loss: 0.7162, Train MPP Accuracy: 0.5256, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6927, Valid MPP Accuracy: 0.5750, Valid ML Accuracy: 0.5750
Epoch [3] Train Totol Loss: 0.7062, Train MPP Accuracy: 0.5128, Train ML Accuracy: 0.4615, Valid Total Loss: 0.7213, Valid MPP Accuracy: 0.5750, Valid ML Accuracy: 0.4250
Epoch [4] Train Totol Loss: 0.7180, Train MPP Accuracy: 0.4872, Train ML Accuracy: 0.5000, Valid Total Loss: 0.7067, Valid MPP Accuracy: 0.5750, Valid ML Accuracy: 0.4250
Epoch [5] Train Totol Loss: 0.7174, Train MPP Accuracy: 0.4487, Train ML Accuracy: 0.4872, Valid Total Loss: 0.6900, Valid MPP Accuracy: 0.5750, Valid ML Accuracy: 0.4250
Epoch [6] Train Totol Loss: 0.7060, Train MPP Accuracy: 0.5128, Train ML Accuracy: 0.4872, Valid Total Loss: 0.6868, Valid MPP Accuracy: 0.5750, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Epoch [36] Train Totol Loss: 0.6918, Train MPP Accuracy: 0.5769, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6975, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5750
Epoch [37] Train Totol Loss: 0.6917, Train MPP Accuracy: 0.5769, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6975, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5750
Epoch [38] Train Totol Loss: 0.6917, Train MPP Accuracy: 0.5769, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6975, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5750
Epoch [39] Train Totol Loss: 0.6916, Train MPP Accuracy: 0.5769, Train ML Accuracy: 0.5000, Valid Total Loss: 0.6976, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5750
Epoch [40] Train Totol Loss: 0.6916, Train MPP Accuracy: 0.5769, Train ML Accuracy: 0.5128, Valid Total Loss: 0.6976, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5750
Epoch [41] Train Totol Loss: 0.6915, Train MPP Accuracy: 0.5769, Train ML Accuracy: 0.5256, Valid Total Loss: 0.6976, Valid MPP Accuracy: 0.

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.9420, Train MPP Accuracy: 0.4808, Train ML Accuracy: 0.4744, Valid Total Loss: 0.7032, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.4000
Epoch [2] Train Totol Loss: 0.7074, Train MPP Accuracy: 0.5321, Train ML Accuracy: 0.5064, Valid Total Loss: 0.6872, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.6000
Epoch [3] Train Totol Loss: 0.7165, Train MPP Accuracy: 0.4808, Train ML Accuracy: 0.4936, Valid Total Loss: 0.6939, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.6000
Epoch [4] Train Totol Loss: 0.7103, Train MPP Accuracy: 0.5064, Train ML Accuracy: 0.4423, Valid Total Loss: 0.7060, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.6000
Epoch [5] Train Totol Loss: 0.7070, Train MPP Accuracy: 0.5449, Train ML Accuracy: 0.4423, Valid Total Loss: 0.6958, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.6000
Epoch [6] Train Totol Loss: 0.7018, Train MPP Accuracy: 0.5705, Train ML Accuracy: 0.4679, Valid Total Loss: 0.6922, Valid MPP Accuracy: 0.5000, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 1.0202, Train MPP Accuracy: 0.5321, Train ML Accuracy: 0.4808, Valid Total Loss: 0.7306, Valid MPP Accuracy: 0.4500, Valid ML Accuracy: 0.5250
Epoch [2] Train Totol Loss: 0.7274, Train MPP Accuracy: 0.4808, Train ML Accuracy: 0.5128, Valid Total Loss: 0.6991, Valid MPP Accuracy: 0.5500, Valid ML Accuracy: 0.5250
Epoch [3] Train Totol Loss: 0.7041, Train MPP Accuracy: 0.5321, Train ML Accuracy: 0.4872, Valid Total Loss: 0.7095, Valid MPP Accuracy: 0.5500, Valid ML Accuracy: 0.5250
Epoch [4] Train Totol Loss: 0.7034, Train MPP Accuracy: 0.5577, Train ML Accuracy: 0.4615, Valid Total Loss: 0.6951, Valid MPP Accuracy: 0.5500, Valid ML Accuracy: 0.5250
Epoch [5] Train Totol Loss: 0.6986, Train MPP Accuracy: 0.5577, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6913, Valid MPP Accuracy: 0.5500, Valid ML Accuracy: 0.5250
Epoch [6] Train Totol Loss: 0.6955, Train MPP Accuracy: 0.5577, Train ML Accuracy: 0.4744, Valid Total Loss: 0.6918, Valid MPP Accuracy: 0.5500, 

In [28]:
model.load_state_dict(torch.load('double_value_cls.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>

In [29]:
def generate_test_data(post1,post2,max_len):
  
  
  
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

In [30]:
 
class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']

      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.first_sen_len[index],self.second_sen_len[index]

In [31]:
def generate_test_dataloader(post1,post2,max_len,batch_size):
  data = generate_test_data(post1,post2,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader

In [32]:
max_len = 256
batch_size = 8
test_post1 = df2['post1']
test_post2 = df2['post2']
   
test_dataloader = generate_test_dataloader(test_post1,test_post2,max_len,batch_size)

In [33]:
def real_prediction(model, 
  test_dataloader,  
  optimizer,
  device
  ):
  
  test_mpp_predictions = []
  test_ml_predictions = []
  real_test_mpp = []
  real_test_ml= []

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        
        first_sen_len = data[3].to(device)
        second_sen_len = data[4].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )

        for i in output_cls:
          if i[0]>0.5:
            test_mpp_predictions.append(1)
          else:
            test_mpp_predictions.append(0)
          if i[1]>0.5:
            test_ml_predictions.append(1)
          else:
            test_ml_predictions.append(0)

  return test_mpp_predictions,test_ml_predictions
      



In [34]:
test_mpp_predictions,test_ml_predictions = real_prediction(model,test_dataloader,optimizer,device)

In [35]:
df2['MPP_prediction'] = test_mpp_predictions
df2['ML_prediction'] = test_ml_predictions
df2.to_json('ERAI_pairwise_[aiml]_[1].json')